In [ ]:
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments
import torch
import pandas as pd

In [4]:
data = pd.read_csv("drugLibTrain_raw.tsv", sep="\t")

In [5]:
data.columns

Index(['Unnamed: 0', 'urlDrugName', 'rating', 'effectiveness', 'sideEffects',
       'condition', 'benefitsReview', 'sideEffectsReview', 'commentsReview'],
      dtype='object')

In [ ]:
df = pd.read_csv("drugLibTrain_raw.tsv", sep="\t")
df = df[['commentsReview', 'rating']].dropna()
df['label'] = df['rating'].apply(lambda x: 1 if x >= 7 else 0)

In [6]:

data_ = data[['commentsReview', 'rating']].dropna()

## condition apply for 7 is positive and else negative values

sentiments = []
for data_rate in data['rating']:
    if data_rate >= 7:
        sentiments.append(1)
    else :
        sentiments.append(0)
data['label'] = sentiments

In [ ]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
from sklearn.model_selection import train_test_split
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)
from transformers import TrainingArguments
from torch.utils.data import TensorDataset
from sklearn.metrics import accuracy_score, f1_score


train_texts, val_texts, train_labels, val_labels = train_test_split(df['commentsReview'], df['label'], test_size=0.2)
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(list(val_texts), truncation=True, padding=True, max_length=512)

train_dataset = {
    'input_ids': torch.tensor(train_encodings['input_ids']),
    'attention_mask': torch.tensor(train_encodings['attention_mask']),
    'labels': torch.tensor(list(train_labels))
}
val_dataset = {
    'input_ids': torch.tensor(val_encodings['input_ids']),
    'attention_mask': torch.tensor(val_encodings['attention_mask']),
    'labels': torch.tensor(list(val_labels))
}

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps", 
)

train_ds = TensorDataset(train_dataset['input_ids'], train_dataset['attention_mask'], train_dataset['labels'])
val_ds = TensorDataset(val_dataset['input_ids'], val_dataset['attention_mask'], val_dataset['labels'])


def compute_metrics(p):
    preds = p.predictions.argmax(-1)
    labels = p.label_ids
    return {
        'accuracy': accuracy_score(labels, preds),
        'f1': f1_score(labels, preds)
    }

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)
trainer.train()

### save model for streamlit run

In [ ]:
model.save_pretrained("sentiment_model")
tokenizer.save_pretrained("sentiment_model")